In [0]:
!pip3 install pulp==1.6.8

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import random
import pulp

In [0]:
# PuLP class for vehicle routing

class CVRP_SPD:
    def __init__(self, numberOfCustomers, numberOfVehicles, capacityOfVehicle, pickups, delivery, costMatrix):
        self.numberOfCustomers = numberOfCustomers
        self.numberOfVehicles  = numberOfVehicles
        self.capacityOfVehicle = capacityOfVehicle
        self.pickups = pickups
        self.delivery = delivery
        self.costMatrix        = costMatrix
        self.initializeLP()
    
    def initializeLP(self):
        self.cvrpLP = pulp.LpProblem("CVRP_PD", pulp.LpMinimize)
        objective = None
        x, y, z = [], [], []
        D, d = [], []
        P, p = [], []
        
        K = self.numberOfVehicles
        N = len(costMatrix)

        # create variables
        for i in range(N):
          xTemp1 = []
          for j in range(N):
            xTemp2 = []
            for k in range(K):
              xTemp3 = pulp.LpVariable('x('+str(i)+','+str(j)+','+str(k)+')', cat='Binary')
              xTemp2.append(xTemp3)
            xTemp1.append(xTemp2)
          x.append(xTemp1)
        
        for i in range(N):
          yTemp1 = []
          for j in range(N):
            yTemp2 = pulp.LpVariable('y('+str(i)+','+str(j)+')', lowBound=0, cat='Continuous')
            yTemp1.append(yTemp2)
          y.append(yTemp1)
        
        for i in range(N):
          zTemp1 = []
          for j in range(N):
            zTemp2 = pulp.LpVariable('z('+str(i)+','+str(j)+')', lowBound=0, cat='Continuous')
            zTemp1.append(zTemp2)
          z.append(zTemp1)
        

        # objective function
        for i in range(N):
          for j in range(N):
            for k in range(K):
            #   print(self.costMatrix[i][j])
            #   print(x[i][j][v])
              objective += self.costMatrix[i][j] * x[i][j][k]
        self.cvrpLP += objective

        #constraints

        #constraint-1
        for i in range(1, N):
          constraint1 = None
          for j in range(N):
            for k in range(K):
              constraint1 += x[i][j][k]
          self.cvrpLP += constraint1 == 1
        
        #constraint-2
        for k in range(K):
            for i in range(N):
                constraint2a, constraint2b = None, None
                for j in range(N):
                    constraint2a += x[i][j][k]
                    constraint2b += x[j][i][k]
                self.cvrpLP += constraint2a - constraint2b == 0
        
        #constraint-3
        for k in range(K):
          constraint3 = None
          for i in range(N):
            constraint3 += x[0][i][k]
          self.cvrpLP += constraint3 <= 1
        
        # constraint-4
        for i in range(N):
            for j in range(N):
                constraint4a = None
                for k in range(K):
                    constraint4a += x[i][j][k]
                self.cvrpLP += y[i][j] + z[i][j] <= self.capacityOfVehicle * constraint4a
        
        #constraint-5
        for i in range(1, N):
            constraint5a, constraint5b = None, None
            for j in range(N):
                constraint5a += y[i][j]
                constraint5b += y[j][i]
            self.cvrpLP += constraint5a - constraint5b == self.pickups[i]

        #constraint-6
        for i in range(1, N):
            constraint6a, constraint6b = None, None
            for j in range(N):
                constraint6a += z[j][i]
                constraint6b += z[i][j]
            self.cvrpLP += constraint6a - constraint6b == self.delivery[i]
        
        print(self.cvrpLP)

    def solve(self):
        status = self.cvrpLP.solve()#pulp.solvers.PULP_CBC_CMD(self.cvrpLP))
        print(pulp.LpStatus[self.cvrpLP.status])
    
    def getResult(self):
        print("Objective value: ", pulp.value(self.cvrpLP.objective))
        for v in self.cvrpLP.variables():
            print(v.name, " = ", v.varValue)
        

In [0]:
costMatrix = [[0,9,14,23,32,50,21,49,30,27,35,28,18],
[9,0,21,22,36,52,24,51,36,37,41,30,20],
[14,21,0,25,38,5,31,7,36,43,29,7,6],
[23,22,25,0,42,12,35,17,44,31,31,11,6],
[32,36,38,42,0,22,37,16,46,37,29,13,14],
[50,52,5,12,22,0,41,23,10,39,9,17,16],
[21,24,31,35,37,41,0,26,21,19,10,25,12],
[49,51,7,17,16,23,26,0,30,28,16,27,12],
[30,36,36,44,46,10,21,30,0,25,22,10,20],
[27,37,43,31,37,39,19,28,25,0,20,16,8],
[35,41,29,31,29,9,10,16,22,20,0,10,10],
[28,30,7,11,13,17,25,27,10,16,10,0,10],
[18,20, 6, 6,14,16,12,12,20,8, 10,10,0]]

delivery = [0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700, 1100]
pickups = [1100, 0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700]
capacityOfVehicle = 6000
numberOfVehicles = 4
numberOfCustomers = len(costMatrix) - 1

In [0]:
lp = CVRP_SPD(numberOfCustomers, numberOfVehicles, capacityOfVehicle, pickups, delivery, costMatrix)
lp.solve()
lp.getResult()

CVRP_PD:
MINIMIZE
9*x(0,1,0) + 9*x(0,1,1) + 9*x(0,1,2) + 9*x(0,1,3) + 35*x(0,10,0) + 35*x(0,10,1) + 35*x(0,10,2) + 35*x(0,10,3) + 28*x(0,11,0) + 28*x(0,11,1) + 28*x(0,11,2) + 28*x(0,11,3) + 18*x(0,12,0) + 18*x(0,12,1) + 18*x(0,12,2) + 18*x(0,12,3) + 14*x(0,2,0) + 14*x(0,2,1) + 14*x(0,2,2) + 14*x(0,2,3) + 23*x(0,3,0) + 23*x(0,3,1) + 23*x(0,3,2) + 23*x(0,3,3) + 32*x(0,4,0) + 32*x(0,4,1) + 32*x(0,4,2) + 32*x(0,4,3) + 50*x(0,5,0) + 50*x(0,5,1) + 50*x(0,5,2) + 50*x(0,5,3) + 21*x(0,6,0) + 21*x(0,6,1) + 21*x(0,6,2) + 21*x(0,6,3) + 49*x(0,7,0) + 49*x(0,7,1) + 49*x(0,7,2) + 49*x(0,7,3) + 30*x(0,8,0) + 30*x(0,8,1) + 30*x(0,8,2) + 30*x(0,8,3) + 27*x(0,9,0) + 27*x(0,9,1) + 27*x(0,9,2) + 27*x(0,9,3) + 9*x(1,0,0) + 9*x(1,0,1) + 9*x(1,0,2) + 9*x(1,0,3) + 41*x(1,10,0) + 41*x(1,10,1) + 41*x(1,10,2) + 41*x(1,10,3) + 30*x(1,11,0) + 30*x(1,11,1) + 30*x(1,11,2) + 30*x(1,11,3) + 20*x(1,12,0) + 20*x(1,12,1) + 20*x(1,12,2) + 20*x(1,12,3) + 21*x(1,2,0) + 21*x(1,2,1) + 21*x(1,2,2) + 21*x(1,2,3) + 22*x(1,3,0) + 